# Deploy COA Objects in Transcend:   DB_Objects (including CDS)

In [ ]:
%addconnect name=Transcend, user=sh186014, host=tdprdcop3.td.teradata.com, props="LOGMECH=LDAP"

In [ ]:
%addconnect name=TranscendAWS, user=sh186014, host=tdprd2cop3.td.teradata.com, props="LOGMECH=LDAP"

In [ ]:
%connect Transcend

# Perm Space (CDS) per week

In [ ]:
Drop Table adlste_coa_stg.stg_dat_permspace_week 

In [ ]:
-- GLOBAL TEMP TABLE:
CREATE MULTISET GLOBAL TEMPORARY TABLE adlste_coa_stg.stg_dat_permspace_week 
(
    Site_ID         VARCHAR(128)   CHARACTER SET UNICODE NOT CASESPECIFIC
  , Week_ID         INTEGER
  , DatabaseName    VARCHAR(128) CHARACTER SET UNICODE NOT CASESPECIFIC
  , Spool_Pct       DECIMAL(4,3)
  , CommentString   VARCHAR(61)  CHARACTER SET UNICODE NOT CASESPECIFIC
  , MaxPerm_GB      DECIMAL(18,6)
  , CurrPerm_GB     DECIMAL(18,6)
  , Fill_Pct        DECIMAL(18,6)
  , CurrPerm_Rank   INTEGER
) NO PRIMARY INDEX 
  ON COMMIT PRESERVE ROWS;

In [ ]:
rename table adlste_coa.coat_dat_permspace_week to adlste_coa.coat_dat_permspace_week_bkup

In [ ]:
CREATE MULTISET TABLE adlste_coa.coat_dat_permspace_week 
(
    Site_ID         VARCHAR(128)   CHARACTER SET UNICODE NOT CASESPECIFIC
  , Week_ID         INTEGER
  , DatabaseName    VARCHAR(128) CHARACTER SET UNICODE NOT CASESPECIFIC
  , Spool_Pct       DECIMAL(4,3)
  , CommentString   VARCHAR(61)  CHARACTER SET UNICODE NOT CASESPECIFIC
  , MaxPerm_GB      DECIMAL(18,6)
  , CurrPerm_GB     DECIMAL(18,6)
  , Fill_Pct        DECIMAL(18,6)
  , CurrPerm_Rank   INTEGER
  , Run_ID           INTEGER
) PRIMARY INDEX (Site_ID, Week_ID)
;

In [ ]:
insert into adlste_coa.coat_dat_permspace_week 
select * from adlste_coa.coat_dat_permspace_week_bkup

In [ ]:
drop table adlste_coa.coat_dat_permspace_week_bkup

In [ ]:
replace view adlste_coa.coa_dat_permspace_week
as  locking row for access
select * from adlste_coa.coat_dat_permspace_week

In [ ]:
REPLACE PROCEDURE adlste_coa.sp_dat_permspace_week ( spversion VARCHAR(128) ) 
SQL SECURITY CREATOR
BEGIN 

    /* start setup audit process */
    
    DECLARE runid  INTEGER;
    DECLARE siteid VARCHAR(128);
    DECLARE tablename VARCHAR(128);
    DECLARE callingsp VARCHAR(128); 
    DECLARE startdate DATE;
    DECLARE enddate DATE;
    
    SET runid = 0;
    SET tablename = 'adlste_coa.coat_dat_permspace_week';
    SET callingsp = 'adlste_coa.sp_dat_permspace_week';
    SET siteid = 'Empty Table';
    SET startdate = DATE;
    SET enddate = DATE;

    SELECT 
     coalesce(Site_ID,'empty table')       
    ,min(substr(trim(Week_ID),1,4)||'-'||substr(trim(Week_ID),5,2)||'-'||
     cast(cast((cast(substr(trim(Week_ID),7,1) as int)*7)-6 as int format'99') as char(2)) )(DATE) as StartDate
    ,max(substr(trim(Week_ID),1,4)||'-'||substr(trim(Week_ID),5,2)||'-'||
     cast(cast((cast(substr(trim(Week_ID),7,1) as int)*7)-6 as int format'99') as char(2)) )(DATE) as EndDate
    INTO siteid, startdate, enddate
    FROM adlste_coa_stg.stg_dat_permspace_week
    Group by 1
    ;
    
    /* end setup audit process */
        
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'START',  '') ;

    DELETE FROM adlste_coa.coat_dat_permspace_week 
    WHERE (Site_ID, Week_ID) IN
    (Select Site_ID, Week_ID From adlste_coa_stg.stg_dat_permspace_week );
    
    Insert into adlste_coa.coat_dat_permspace_week 
    Select 
      Site_ID          
    , Week_ID          
    , DatabaseName   
    , Spool_Pct
    , CommentString
    , MaxPerm_GB
    , CurrPerm_GB
    , Fill_Pct
    , CurrPerm_Rank
    , :runid
    from adlste_coa_stg.stg_dat_permspace_week ; 
    
    delete from adlste_coa_stg.stg_dat_permspace_week;
    
    collect stats column(Site_ID, Week_ID), column(Site_ID) on adlste_coa.coat_dat_permspace_week ;
        
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'END', '') ;

END;

In [ ]:
insert into adlste_coa_stg.stg_dat_permspace_week 
select Site_ID, Week_ID, DatabaseName, Spool_Pct, CommentString, MaxPerm_GB, CurrPerm_GB, Fill_Pct, CurrPerm_Rank
from adlste_coa.coa_dat_permspace_week 
qualify rank()over(order by Site_ID, Week_ID) = 1

In [ ]:
call adlste_coa.sp_dat_permspace_week('test')

In [ ]:
select top 100 * from adlste_coa.coa_audit_log 
where TableName = 'adlste_coa.sp_dat_permspace_week'
order by 1 desc

<br><br><br>
# dat_dbobject_counts

In [ ]:
DROP TABLE adlste_coa_stg.stg_dat_dbobject_counts

In [ ]:
CREATE GLOBAL TEMPORARY TABLE adlste_coa_stg.stg_dat_dbobject_counts
( Site_ID        VARCHAR(128) CHARACTER SET LATIN
, LogDate        DATE FORMAT 'YYYY/MM/DD'
, DBName         VARCHAR(128) CHARACTER SET LATIN 
, Table_Count     INTEGER
, View_Count      INTEGER
, Index_Count     INTEGER
, Macro_Count     INTEGER
, SPTrig_Count    INTEGER
, UDObject_Count  INTEGER
, Other_Count     INTEGER
) NO PRIMARY INDEX 
ON COMMIT PRESERVE ROWS;

In [ ]:
DROP TABLE adlste_coa.coat_dat_dbobject_counts

In [ ]:
CREATE TABLE adlste_coa.coat_dat_dbobject_counts
( Site_ID        VARCHAR(128) CHARACTER SET LATIN
, LogDate        DATE FORMAT 'YYYY/MM/DD'
, DBName         VARCHAR(128) CHARACTER SET LATIN 
, Table_Count    INTEGER
, View_Count     INTEGER
, Index_Count    INTEGER
, Macro_Count    INTEGER
, SPTrig_Count   INTEGER
, UDObject_Count INTEGER
, Other_Count    INTEGER
, Run_ID         INTEGER
) PRIMARY INDEX (LogDate, DBName);

In [ ]:
replace view adlste_coa.coa_dat_dbobject_counts as
locking row for access
select * from adlste_coa.coat_dat_dbobject_counts

In [ ]:
REPLACE PROCEDURE adlste_coa.sp_dat_dbobject_counts (spversion VARCHAR(128)) 
SQL SECURITY CREATOR
BEGIN 

    /* start setup audit process */
    DECLARE runid  INTEGER;
    DECLARE runtype VARCHAR(128);
    DECLARE startdate DATE;
    DECLARE enddate DATE;
    DECLARE siteid VARCHAR(128);
    DECLARE procname VARCHAR(128);
    DECLARE tablename VARCHAR(128);
    DECLARE callingsp VARCHAR(128);
    DECLARE rowcount bigint;

    SET procname = 'dat_dbobject_counts';
    SET callingsp = 'adlste_coa.sp_' || procname;
    SET tablename = 'adlste_coa.coat_' || procname;
    -------------
    SET runid = 0;
    SET siteid = 'All';
    SET runtype = 'COA Run';

    SET startdate = DATE;
    SET enddate = DATE;

    SELECT 
     coalesce(Site_ID,'empty table')       
    ,min(LogDate)(DATE) as StartDate
    ,max(LogDate)(DATE) as EndDate
    INTO siteid, startdate, enddate
    FROM adlste_coa_stg.stg_dat_dbobject_counts
    Group by 1
    ;
    
    CALL adlste_coa.sp_get_rowcount(:tablename, rowcount);
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate, :runtype,
        'START', 'record count: ' || cast(rowcount as varchar(32)) ) ;   /*  run_status, run_msg */
    /* ------------------------
       end setup, begin sp logic
       ------------------------
    */
 
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate, :runtype, 
        'STEP 01', 'Delete Duplicates from Target') ;   
        DELETE FROM adlste_coa.coat_dat_dbobject_counts 
        WHERE (Site_ID, LogDate) IN
        (Select Site_ID, LogDate From adlste_coa_stg.stg_dat_dbobject_counts );

    
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion,  :startdate, :enddate, :runtype,
        'STEP 02', 'Insert into Target') ;   
        Insert into adlste_coa.coat_dat_dbobject_counts 
        Select 
          Site_ID          
        , LogDate
        , DBName       
        , Table_Count   
        , View_Count    
        , Index_Count   
        , Macro_Count   
        , SPTrig_Count
        , UDObject_Count
        , Other_Count   
        , :runid
        from adlste_coa_stg.stg_dat_dbobject_counts ; 
    
    
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion,  :startdate, :enddate, :runtype,
        'STEP 03', 'Delete from GTT') ;   
        delete from adlste_coa_stg.stg_dat_dbobject_counts;
                        
            
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion,  :startdate, :enddate, :runtype,
            'STEP 04', 'Collect Stats') ;   
        collect stats column(Site_ID, LogDate), column(Site_ID) on adlste_coa.coat_dat_dbobject_counts;
            
    /* ------------------------
       end sp logic, begin clean-up
       ------------------------
    */   
    CALL adlste_coa.sp_get_rowcount(:tablename, rowcount);
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion,  :startdate, :enddate, :runtype,
        'END', 'record count: ' || cast(rowcount as varchar(32)) ) ;

END;


In [ ]:
delete from adlste_coa_stg.stg_dat_dbobject_counts all;
insert into adlste_coa_stg.stg_dat_dbobject_counts 
select 'WestTest', LogDate, DBName, TableCount, ViewCount, IndexCount, MacroCount, "SP&TrigCount", UDObjectCount, OtherCount
from  adlste_coa.coa_dat_dbobject_counts ;

In [ ]:
CALL adlste_coa.sp_dat_dbobject_counts('v1');

In [ ]:
Select top 10 * from adlste_coa.coa_dat_dbobject_counts where LogDate =DATE and Site_ID = 'WestTest'

In [ ]:
-- Check Audit Logs:
select case when ParentRun_ID=0 then Run_ID else ParentRun_ID end as RunID
,TableName, CallingSP, activity_cnt, Run_Status, Run_Message, Run_User, Run_TS
from adlste_coa.coa_audit_runlog
where  CallingSP = 'adlste_coa.sp_dat_dbobject_counts' and Site_ID = 'Transcend02'
qualify Parentrun_id = max(ParentRun_ID)over(partition by Site_ID)
     or run_id = max(ParentRun_ID)over(partition by Site_ID)
order by ParentRun_ID asc, Run_ID 

<br><br><br>
# DB_Objects including CDS

This used to be one table structure, containing both CDS (can be measured over time in pdcr) and db object counts (always point-in-time).   Because of the different time dimensions, we pulled apart into 2 tables:
- coa_dat_permspace_week
- coa_dat_objectkind_count

Because there are still powerBI reports on this view, we have changed this process to a view combining the above tow structures, where available. 

In [ ]:
-- PERM Table  just for reference...
/*
CREATE MULTISET TABLE adlste_coa.coat_dat_DB_Objects 
(  Site_ID          VARCHAR(128)  CHARACTER SET LATIN
 , Week_ID          INTEGER
 , DatabaseName     VARCHAR(128)  CHARACTER SET UNICODE 
 , CurrPermGB_Rank  INTEGER       COMPRESS(0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250)
 , CommentString    VARCHAR(1024) CHARACTER SET UNICODE 
 , MaxPermGB        DECIMAL(18,2) COMPRESS(0) 
 , CurrentPermGB    DECIMAL(18,2) COMPRESS(0) 
 , FilledPct        DECIMAL(9,4)  COMPRESS(0.0000)
 , Table_Count      INTEGER       COMPRESS(0) 
 , View_Count       INTEGER       COMPRESS(0)
 , Index_Count      INTEGER       COMPRESS(0) 
 , Macro_Count      INTEGER       COMPRESS(0) 
 , SPTrig_Count     INTEGER       COMPRESS(0)  
 , UDObject_Count   INTEGER       COMPRESS(0)    
 , Other_Count      INTEGER       COMPRESS(0) 
 , Run_ID           INTEGER
 ) PRIMARY INDEX (Site_ID, Week_ID, DatabaseName --)-- break syntax (just in case... don't run)
 */

In [ ]:
replace view adlste_coa.coa_dat_DB_Objects
as  locking row for access

Select 
  cds.Site_ID
, cds.Week_ID
, cds.DatabaseName
, cds.CurrPerm_Rank as CurrPermGB_Rank
--, Spool_Pct
, cds.CommentString
, cds.MaxPerm_GB as MaxPermGB
, cds.CurrPerm_GB as CurrentPermGB
, cds.Fill_Pct as FilledPct
, obj.Table_Count 
, obj.View_Count 
, obj.Index_Count 
, obj.Macro_Count 
, obj.SPTrig_Count
, obj.UDObject_Count 
, obj.Other_Count 
, case when cds.Run_ID > obj.Run_ID then cds.Run_ID else obj.Run_ID end as Run_ID

from adlste_coa.coa_dat_permspace_week as cds
left outer 
join (Select Site_ID, DBName
      ,(cast(YearNumber_of_Calendar(LogDate,'ISO') as int)*1000) +
       (cast(MonthNumber_of_Year   (LogDate,'ISO') as int)*10) +
       (cast(WeekNumber_of_Month   (LogDate,'ISO') as int)) as Week_ID
      , avg(Table_Count)(INT)  as Table_Count
      , avg(View_Count)(INT)  as View_Count
      , avg(Index_Count)(INT)  as Index_Count
      , avg(Macro_Count)(INT)  as Macro_Count
      , avg(SPTrig_Count)(INT)  as SPTrig_Count
      , avg(UDObject_Count)(INT)  as UDObject_Count
      , avg(Other_Count)(INT)  as Other_Count
      , max(Run_ID) as Run_ID
     FROM adlste_coa.coa_dat_dbobject_counts 
     GROUP BY Site_ID, DBName, Week_ID) as obj 
 on cds.Site_ID = obj.Site_ID 
and cds.DatabaseName = obj.DBName
and cds.Week_ID = (obj.Week_ID)      
-- this needs some love

In [ ]:
select * from adlste_coa.coa_dat_DB_Objects

<br/><br/><br/><br/>
# Column Type

In [ ]:
drop table adlste_coa_stg.stg_dat_column_type ;

In [ ]:
CREATE MULTISET GLOBAL TEMPORARY TABLE adlste_coa_stg.stg_dat_column_type 
( Site_ID          VARCHAR(128) CHARACTER SET LATIN
, LogDate          DATE 
, Column_Type      VARCHAR(256) CHARACTER SET LATIN 
, Column_Category  VARCHAR(128) CHARACTER SET LATIN 
, Total_Cnt        INT
)
NO PRIMARY INDEX 
ON COMMIT PRESERVE ROWS;

In [ ]:
drop table adlste_coa.coat_dat_column_type

In [ ]:
CREATE MULTISET TABLE adlste_coa.coat_dat_column_type 
( Site_ID          VARCHAR(128) CHARACTER SET LATIN
, LogDate          DATE
, Column_Type      VARCHAR(256) CHARACTER SET LATIN 
, Column_Category  VARCHAR(128) CHARACTER SET LATIN 
, Total_Cnt        INT
, Run_ID           INT
)
PRIMARY INDEX (Site_ID, LogDate);

In [ ]:
replace view adlste_coa.coa_dat_column_type
as locking row for access
select * from adlste_coa.coat_dat_column_type

In [ ]:
REPLACE PROCEDURE adlste_coa.sp_dat_column_type ( spversion VARCHAR(128) ) 
SQL SECURITY CREATOR
BEGIN 

    /* start setup audit process */
    
    DECLARE runid  INTEGER;
    DECLARE siteid VARCHAR(128);
    DECLARE tablename VARCHAR(128);
    DECLARE callingsp VARCHAR(128); 
    DECLARE startdate DATE;
    DECLARE enddate DATE;
    
    SET runid = 0;
    SET tablename = 'adlste_coa.coat_dat_column_type';
    SET callingsp = 'adlste_coa.sp_dat_column_type';
    SET siteid = 'Empty Table';
    SET startdate = DATE;
    SET enddate = DATE;

    SELECT 
     coalesce(Site_ID,'empty table')       
    ,min(LogDate) as StartDate
    ,max(LogDate) as EndDate
    INTO siteid, startdate, enddate
    FROM adlste_coa_stg.stg_dat_column_type
    Group by 1
    ;
    
    /* end setup audit process */
    
    
    
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'START',  '') ;

    DELETE FROM adlste_coa.coat_dat_column_type 
    WHERE (Site_ID, LogDate) IN
    (Select Site_ID, LogDate From adlste_coa_stg.stg_dat_column_type );
    
    Insert into adlste_coa.coat_dat_column_type 
    Select
     Site_ID
    ,LogDate
    ,Column_Type
    ,Column_Category
    ,Total_Cnt
    , :runid
    from adlste_coa_stg.stg_dat_column_type as a ; 
    
    delete from adlste_coa_stg.stg_dat_column_type;
        
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'END', '') ;

END;

In [ ]:
select * from adlste_coa.coat_dat_column_type

In [ ]:
select top 100 * from adlste_coa.coa_audit_log 
where TableName = 'adlste_coa.coat_dat_column_type'
order by 1 desc

<br/><br/><br/><br/>
# Other sql

In [ ]:
-- return set, multiset, and other table types
/*{{save:table_set_type.csv}}*/
SELECT '{siteid}' as Site_ID
,sum(case when CheckOpt = 'Y' then 1 else 0 end) as MultiSet_Cnt
,sum(case when CheckOpt = 'N' then 1 else 0 end) as Set_Cnt
,sum(case when CheckOpt not in('Y','N') then 1 else 0 end) as Other_Cnt
FROM DBC.Tables

 

In [ ]:

CREATE VOLATILE TABLE FeatureDataType AS -- column datatypes
    (
        SELECT
            '++' AS DataType
            ,CAST('TD_ANYTYPE' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'A1' AS DataType
            ,CAST('ARRAY (one dimensional)' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'AN' AS DataType
            ,CAST('ARRAY (multidimensional)' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'AT' AS DataType
            ,CAST('TIME' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'BF' AS DataType
            ,CAST('BYTE' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'BN' AS DataType
            ,CAST('BSON' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'BO' AS DataType
            ,CAST('BINARY LARGE OBJECT' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'BV' AS DataType
            ,CAST('BYTE VARYING' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'CF' AS DataType
            ,CAST('CHARACTER (fixed)' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'CO' AS DataType
            ,CAST('CHARACTER LARGE OBJECT' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'CV' AS DataType
            ,CAST('CHARACTER (varying)' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'D' AS DataType
            ,CAST('DECIMAL' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'DA' AS DataType
            ,CAST('DATE' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'DH' AS DataType
            ,CAST('INTERVAL DAY TO HOUR' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'DM' AS DataType
            ,CAST('INTERVAL DAY TO MINUTE' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'DS' AS DataType
            ,CAST('INTERVAL DAY TO SECOND' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'DY' AS DataType
            ,CAST('INTERVAL DAY' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'F' AS DataType
            ,CAST('FLOAT' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'HM' AS DataType
            ,CAST('INTERVAL HOUR TO MINUTE' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'HR' AS DataType
            ,CAST('INTERVAL HOUR' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'HS' AS DataType
            ,CAST('INTERVAL HOUR TO SECOND' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'I' AS DataType
            ,CAST('INTEGER' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'I1' AS DataType
            ,CAST('BYTEINT' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'I2' AS DataType
            ,CAST('SMALLINT' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'I8' AS DataType
            ,CAST('BIGINT' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'JN' AS DataType
            ,CAST('JSON' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'MI' AS DataType
            ,CAST('INTERVAL MINUTE' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'MO' AS DataType
            ,CAST('INTERVAL MONTH' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'MS' AS DataType
            ,CAST('INTERVAL MINUTE TO SECOND' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'N' AS DataType
            ,CAST('NUMBER' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'PD' AS DataType
            ,CAST('PERIOD(DATE)' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'PM' AS DataType
            ,CAST('PERIOD(TIMESTAMP(n) WITH TIME ZONE)' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'PS' AS DataType
            ,CAST('PERIOD(TIMESTAMP(n))' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'PT' AS DataType
            ,CAST('PERIOD(TIME(n))' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'PZ' AS DataType
            ,CAST('PERIOD(TIME(n) WITH TIME ZONE)' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'SC' AS DataType
            ,CAST('INTERVAL SECOND' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'SZ' AS DataType
            ,CAST('TIMESTAMP WITH TIME ZONE' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'TS' AS DataType
            ,CAST('TIMESTAMP' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'TZ' AS DataType
            ,CAST('TIME WITH TIME ZONE' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'UN' AS DataType
            ,CAST('UBJSON' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'UT' AS DataType
            ,CAST('USER-DEFINED TYPE (all types)' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'XM' AS DataType
            ,CAST('XML' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'YM' AS DataType
            ,CAST('INTERVAL YEAR TO MONTH' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
        UNION
        SELECT
            'YR' AS DataType
            ,CAST('INTERVAL YEAR' AS VARCHAR(50)) AS DataTypeDesc
        FROM
            DBC.DBCINFO
    )
    WITH DATA
    UNIQUE PRIMARY INDEX (DataType)
    ON COMMIT PRESERVE ROWS;

COLLECT STATISTICS ON FeatureDataType INDEX (DataType);

In [ ]:
select * from FeatureDataType